In [13]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy
import seaborn as sns
from datetime import timedelta, date
import datetime
from itertools import *

In [14]:
df = pd.read_csv('spy_quotedata.csv', sep='\s+', header=None, skiprows=0) 


In [17]:
df.columns = ['1stCol']


spot_price = df['1stCol'][0].split(',')[1]
spot_price = float(spot_price.split('Last:')[1])

ValueError: Length mismatch: Expected axis has 7 elements, new values have 1 elements

In [ ]:

df = df.iloc[3:]

In [10]:
df.head()

,0,1,2,3,4,5,6
0,SPDR,S&P,500,ETF,"Trust,Last:","388.55,Change:",2.99
1,"Date: 19 September 2022, 8:29 PM GMT-4,Bid:","389.76,Ask:","389.8,Size:","1*6,""Volume:","7,31,75,183""",NaN,NaN
2,Expiration,"Date,Calls,Last","Sale,Net,Bid,Ask,Volume,IV,Delta,Gamma,Open","Interest,Strike,Puts,Last","Sale,Net,Bid,Ask,Volume,IV,Delta,Gamma,Open",Interest,NaN
3,Mon,Oct,03,"2022,SPY221003C00385000,10.2,1.41,9.79,9.84,11...",NaN,NaN,NaN
4,Mon,Oct,03,"2022,SPY221003C00386000,9.59,1.365,9.17,9.22,5...",NaN,NaN,NaN


In [ ]:
new = df["1stCol"].str.split(",", n = 21, expand = True)
new.columns = ['ExpirationDate','Calls','CallLastSale','CallNet','CallBid','CallAsk','CallVol','CallIV','CallDelta','CallGamma','CallOpenInt','CallStrike','Puts','PutLastSale','PutNet','PutBid','PutAsk','PutVol','PutIV','PutDelta','PutGamma','PutOpenInt']

callStrike = [x[-6:-3] for x in new.Calls]
new['StrikePrice'] = callStrike 

new['CallGamma'] = new['CallGamma'].astype(float)
new['CallOpenInt'] = new['CallOpenInt'].astype(float)
new['CallGEX'] = new['CallGamma'] * new['CallOpenInt'] * 100 * spot_price

new['PutGamma'] = new['PutGamma'].astype(float)
new['PutOpenInt'] = new['PutOpenInt'].astype(float)
new['PutGEX'] = new['PutGamma'] * new['PutOpenInt'] * 100 * spot_price * -1



new['TotalGamma'] = new.CallGEX + new.PutGEX
count = 0
strikeWithGamma = []
for a, b in zip(new.StrikePrice, new.TotalGamma):
    strikesPlusGamma = (a,b)
    strikeWithGamma.append(strikesPlusGamma)
new['StrikeAndGamma'] = strikeWithGamma
new = new[(new['TotalGamma'] != 0.0)]

new.sort_values('StrikePrice').plot(x = 'StrikePrice', y = 'TotalGamma', grid = True)
plt.show()

def zero_gex(strikes):
    def add(a, b):
        return (b[0], a[1] + b[1])

    cumsum = list(accumulate(strikes, add))
    if cumsum[len(strikes) // 10][1] < 0:
        op = min
    else:
        op = max
    return op(cumsum, key=lambda i: i[1])[0]

print(zero_gex(new.StrikeAndGamma))